
# Goal
The goal in this competition is to take an image of a handwritten single digit, and determine what that digit is.
For every in the test set, you should predict the correct label.
# Metric
This competition is evaluated on the categorization accuracy of your predictions (the percentage of images you get correct).

# Submission File Format
The file should contain a header and have the following format:

ImageId,Label
* 1,0
* 2,0
* 3,0
* etc.


在这个程序中，我们使用已经被卷积核锐化过的数据集结合CNN进行预测。

In [20]:
import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import warnings
# filter warnings
warnings.filterwarnings('ignore')

In [ ]:
# 先用pandas读入csv
from sklearn.model_selection import train_test_split
df = pd.read_csv('convolution_data.csv') # 导入卷积核锐化后的训练集进行训练
Y = df.iloc[:, 0].values  # 标签数据
X = df.iloc[:, 1:].values  # 特征数据
Y_test = df.iloc[:, 0].values   # 标签数据
X_test = df.iloc[:, 1:].values  # 特征数据
np.save("X.npy", X)
np.save("Y.npy", Y)
X_train = np.load('X.npy')
Y_train = np.load('Y.npy')

# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

print("数据形状:", X_train.shape) 
print("数据形状:", Y_train.shape) 
print("数据形状:", X_test.shape) 
print("数据形状:", Y_test.shape) 

In [ ]:
# load data set
img_size = 28
dataX = np.load('X.npy') # 查看数据形状
print("数据形状:", dataX.shape) # 打印前五行数据 
print(dataX[:10])
dataY = np.load('Y.npy') # 查看数据形状
print("数据形状:", dataY.shape) # 打印前五行数据

print(dataY[:10])

plt.subplot(1, 3, 1)
plt.imshow(dataX[290].reshape(img_size, img_size))
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(dataX[950].reshape(img_size, img_size))
plt.axis('off')
plt.subplot(1, 3, 3)
plt.imshow(dataX[1560].reshape(img_size, img_size))
plt.axis('off')


查看卷积核锐化后的图像，有一定效果，并且保留了图形的绝大部分轮廓

* 为手势零的图像创建标签数组，其值为0，为一手势的图像创建标签数组，其值为1

In [ ]:
Y_test = Y_test.reshape(-1, 1)
Y_train = Y_train.reshape(-1, 1)
print("Y_test shape: ", Y_test.shape)
print("Y_train shape: ", Y_train.shape)
print("X_shape: " , X.shape)
print("Y_shape: " , Y.shape)
print("X_test shape: " , X_test.shape)
print("Y_test shape: " , Y_test.shape)

In [ ]:
# Then lets create x_train, y_train, x_test, y_test arrays
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

number_of_train = X_train.shape[0]
number_of_test = X_test.shape[0]
print("X_train number: " , number_of_train)
print("X_test number: " , number_of_test)

* 现在我们有3维的输入数组（X），为了将其作为感知机模型的输入，我们需要将其展平为2维
* 我们的标签数组（Y）已经是展平的（2D），所以我们保持不变。
* 将X数组（图像数组）展平。


In [25]:
# X_train_flatten = X_train.reshape(number_of_train,X_train.shape[1]*X_train.shape[2])
# X_test_flatten = X_test .reshape(number_of_test,X_test.shape[1]*X_test.shape[2])
# print("X train flatten",X_train_flatten.shape)
# print("X test flatten",X_test_flatten.shape)

* 为方便后续矩阵乘法的计算，对样本矩阵进行转置

In [ ]:
y_train = Y_train.T
y_test = Y_test.T
print("x train: ",X_train.shape)
print("x test: ",X_test.shape)
print("y train: ",y_train.shape)
print("y test: ",y_test.shape)

<!-- <a id="10"></a> <br>
# 深度神经网络(DNN)
* 2-layer 感知机: 
    * 神经网络结构： 包括输入层、一个或多个隐藏层以及输出层。
    * 激活函数： 激活函数提供了非线性变换，使模型具备拟合复杂数据的能力。tanh激活函数比sigmoid更适合用于隐藏单元，因为它的输出均值更接近于零，从而更好地中心化数据以供下一层使用。

<a href="http://ibb.co/eF315x"><img src="http://preview.ibb.co/dajVyH/9.jpg" alt="9" border="0"></a>

   
    
     -->

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 创建卷积神经网络模型
Size_of_Core = 5

model = Sequential(
    [
        # 第一层卷积层：32个5x5的卷积核，激活函数ReLU，填充方式为SAME
        Conv2D(
            32,
            (Size_of_Core, Size_of_Core),
            activation="relu",
            input_shape=(28, 28, 1),
            padding="same",
        ),
        # 第一层池化层：2x2最大池化
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),  # 添加Dropout层，防止过拟合
        # 第二层卷积层：64个5x5的卷积核，激活函数ReLU，填充方式为SAME
        Conv2D(64, (Size_of_Core, Size_of_Core), activation="relu", padding="same"),
        # 第二层池化层：2x2最大池化
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        # 第三层卷积层：128个5x5的卷积核，激活函数ReLU，填充方式为SAME
        Conv2D(128, (Size_of_Core, Size_of_Core), activation="relu", padding="same"),
        # 第三层池化层：2x2最大池化
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        # 展平层，将二维图像数据展平为一维
        Flatten(),
        # 全连接层：128个神经元，激活函数ReLU
        Dense(128, activation="relu"),
        Dropout(0.5),  # 再次添加Dropout层
        # 输出层：10个神经元（对应10个分类），激活函数Softmax
        Dense(10, activation="softmax"),
    ]
)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=10,  # 随机旋转范围
    width_shift_range=0.1,  # 水平移动
    height_shift_range=0.1,  # 垂直移动
    shear_range=0.1,  # 剪切变换程度
    zoom_range=0.1,  # 放大/缩小  
    horizontal_flip=False,  # 水平翻转
    fill_mode="nearest",  # 填充模式
)

# 打印模型摘要，检查各层的输出形状
model.summary()


# 假设你已经有了自己的训练数据 X_train 和 y_train
# 处理现有的训练数据 X_train（假设它是二维的）
X_train = (
    X_train.reshape(X_train.shape[0], 28, 28, 1).astype("float32") / 255.0
)  # 调整为 (num_samples, 28, 28, 1)

# 编译模型
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# 使用数据增强进行训练
datagen.fit(X_train)
model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=20,
    validation_data=(X_test, y_test),  # 用 MNIST 测试集验证
)

# 在测试数据上进行评估
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

相比于之前手写的简单的锐化函数，用卷积核锐化的准确度又有了明显提升。

In [ ]:
test_df = pd.read_csv("test.csv")
test_X = test_df.iloc[:, :].values  
print("数据形状:", test_X.shape)

In [ ]:
test_X_2D=np.array([test_X[i].reshape(28,28) for i in range(28000)])
test_X_2D.shape

下面使用相同的卷积核锐化方式对预测图片集进行锐化

In [ ]:
sharpen_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]) # 这样的卷积核可以强化图像的中心特征

# 假设 dataX_2D 是包含彩色图像的数组，形状为 (28000, 高, 宽, 3)
test_X_2D_convolution = [[] for _ in range(28000)]

for i in range(28000):
    # 确保每张图像是 uint8 类型，且是彩色图像
    image = test_X_2D[i].astype(np.uint8)

    # 对彩色图像应用锐化滤波
    test_X_2D_convolution[i] = cv2.filter2D(image, -1, sharpen_kernel)
test_X_2D_convolution=np.array(test_X_2D_convolution)

plt.subplot(1, 3, 1)
plt.imshow(test_X_2D_convolution[3].reshape(img_size, img_size))
plt.axis("off")
plt.subplot(1, 3, 2)
plt.imshow(test_X_2D_convolution[83].reshape(img_size, img_size))
plt.axis("off")
plt.subplot(1, 3, 3)
plt.imshow(test_X_2D_convolution[1500].reshape(img_size, img_size))
plt.axis("off")

In [ ]:
test_number=test_X_2D_convolution.shape[0]
test_X_2D_convolution = (test_X_2D_convolution.reshape(test_number, 28, 28, 1).astype("float32") / 255.0)
print(test_X_2D_convolution.shape)
print(test_X_2D_convolution[0])


In [ ]:
# 假设 prediction 是模型预测的结果
prediction = model.predict(test_X_2D_convolution)

# 获取最大值的索引作为预测的类别
predicted_class = np.argmax(prediction, axis=-1)

# 将预测结果转换为 DataFrame
output_df = pd.DataFrame(predicted_class, columns=["Label"])

# 添加 ImageId 列，假设它是从 1 开始的
output_df["ImageId"] = output_df.index + 1

# 重新排列列顺序，以确保 'ImageId' 在前
output_df = output_df[["ImageId", "Label"]]

# 将 DataFrame 写入 CSV 文件
output_df.to_csv("output_convolution.csv", index=False, header=True)